<a href="https://colab.research.google.com/github/teddytesfa/Build-AI-CodeAgents-Using-Huggingface-Smolagent-Library/blob/main/smolagents_llm_tool_calling_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Smolagents Tool Calling Demonstration

This notebook showcases calling **external APIs** from HuggingFace smolagents using both the `@tool` decorator and custom subclassing. The example uses the OpenWeatherMap API, but is easily adaptable to other APIs.

Author: Teddy Tesfa

---

## 1. Install Requirements

You need the HuggingFace smolagents library and requests.


In [ ]:
!pip install smolagents requests

## 2. Using the `@tool` Decorator

Define a tool via the decorator for easy integration by LLM agents.


In [3]:
import requests
from smolagents import tool
from google.colab import userdata

@tool
def get_weather(city: str) -> str:
    """
    Retrieves the current weather for a city using OpenWeatherMap.
    Args:
        city (str): Name of the city.
    Returns:
        str: Weather and temperature summary.
    """
    api_key = userdata.get('OPEN_WEATHER_API_KEY') # Replace with your OpenWeatherMap API key
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}"
    resp = requests.get(url)
    data = resp.json()

    if resp.status_code != 200 or "weather" not in data:
        return f"Failed to fetch weather data. Response: {data}"
    return f"The weather in {city} is {data['weather'][0]['description']} with a temperature of {data['main']['temp']}°K."


## 3. Using a Custom Tool Subclass

Subclass the `Tool` class for richer logic, context, or configuration.


In [8]:
from smolagents import Tool

class CustomWeatherTool(Tool):
    name = "get_custom_weather"
    description = "Retrieves current weather for a city using OpenWeatherMap API."
    inputs = {"city": {"type": "string", "description": "The city to look up."}}
    output_type = "string"

    def forward(self, city: str) -> str:
        api_key = userdata.get('OPEN_WEATHER_API_KEY') # Replace with your OpenWeatherMap API key
        url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}"
        resp = requests.get(url)
        data = resp.json()
        if resp.status_code != 200 or "weather" not in data:
            return f"Failed to fetch weather data. Response: {data}"
        return f"The weather in {city} is {data['weather'][0]['description']} with a temperature of {data['main']['temp']}°K."

## 4. Run the Agent with Either Tool

Instantiate a `CodeAgent`, giving it your tool(s) and an LLM model. Then call it:


In [9]:
from smolagents import CodeAgent, InferenceClientModel

# Init model and agent (smolagents supports HF endpoints and local deployments)
model=InferenceClientModel(model_id="Qwen/Qwen3-Coder-30B-A3B-Instruct", provider="nebius", api_key=userdata.get("HF_NEBIUS_API_KEY"))

# Example using the decorator tool
agent = CodeAgent(tools=[get_weather], model=model)
result = agent.run("What's the weather in London?")
print("Decorator tool:", result)

# Example using the custom class tool
weather_tool = CustomWeatherTool()
agent2 = CodeAgent(tools=[weather_tool], model=model)
result2 = agent2.run("Give me the weather in Addis Ababa")
print("Tool subclass:", result2)


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What's the weather in London?                                                                                   │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen3-Coder-30B-A3B-Instruct ──────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  weather = get_weather(city="London")                                                                             
  final_answer(weather)                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: The weather in London is overcast clouds with a temperature of 286.2°K.

[Step 1: Duration 0.88 seconds| Input tokens: 2,070 | Output tokens: 18]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  weather_info = get_custom_weather(city="Addis Ababa")                                                            
  final_answer(weather_info)                                                                                       
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: The weather in Addis Ababa is overcast clouds with a temperature of 286.25°K.

[Step 1: Duration 0.84 seconds| Input tokens: 2,074 | Output tokens: 24]

Tool subclass: The weather in Addis Ababa is overcast clouds with a temperature of 286.25°K.


## 5. Notes

- Replace `OPEN_WEATHER_API_KEY` with your OpenWeatherMap key.
- Both approaches are compatible with other APIs.
- Type hints and docstrings help LLMs understand tool capabilities.
- The agent will automatically use the correct tool if the prompt matches.
